## PART 2

In [10]:
using CSV, DataFrames
filepath = "interpolated_height.csv"
df = CSV.read(filepath, DataFrame)	
H = round.(df[:, 2], digits=0)

317-element Vector{Float64}:
 -300.0
 -138.0
  -35.0
  -33.0
 -131.0
 -288.0
 -126.0
  -23.0
  -21.0
 -120.0
    ⋮
  -26.0
 -112.0
 -290.0
 -118.0
  -46.0
  -93.0
 -273.0
 -140.0
  -67.0

# Problem 3

In [ ]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[1+abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )

    @objective(myModel, Min, sum(x[j] for j=1:h) )

    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -seconds 20 -solve -quit (default strategy 1)
seconds was changed from 1e+100 to 20
Continuous objective value is 68.1555 - 0.02 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0004I processed model has 316 rows, 317 columns (317 integer (317 of which binary)) and 1576 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 281 integers unsatisfied sum - 82.8701
Cbc0038I Pass   1: suminf.   18.83904 (83) obj. 101.659 iterations 242
Cbc0038I Pass   2: suminf.    7.11664 (50) obj. 106.972 iterations 73
Cbc0038I Pass   3: suminf.    6.00408 (47) obj. 107.004 iterations 6
Cbc0038I Pass   4: suminf.    0.52336 (22) obj. 124.523 iterations 30
Cbc0038I Solution found of 145
Cbc0038I Rounding solution of 128 is better than previous of 145

Cbc0038I Befo

# Problem 4

In [3]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[1+abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )
    @variable(myModel, z[1:h] >= 0 )

    @objective(myModel, Min, sum(z[j] for j=1:h) )

    #introduce z for the absolute value of R-H-CHD
    @constraint(myModel, [j=1:h],z[j]>=R[j]-H[j]-10 )
    @constraint(myModel, [j=1:h],z[j]>=-R[j]+H[j]+10 )
    
    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -seconds 20 -solve -quit (default strategy 1)
seconds was changed from 1e+100 to 20
Continuous objective value is 23.8466 - 0.01 seconds
Cgl0003I 0 fixed, 321 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 633 rows, 950 columns (317 integer (317 of which binary)) and 2528 elements
Cbc0038I Initial state - 314 integers unsatisfied sum - 68.4365
Cbc0038I Pass   1: suminf.   66.84759 (312) obj. 480.617 iterations 12
Cbc0038I Pass   2: suminf.   61.30993 (294) obj. 3425.76 iterations 34
Cbc0038I Pass   3: suminf.   53.43069 (267) obj. 9445.47 iterations 58
Cbc0038I Pass   4: suminf.   45.50803 (242) obj. 13456.5 iterations 55
Cbc0038I Pass   5: suminf.   36.00171 (207) obj. 23022.3 iterations 76
Cbc0038I Pass   6: suminf.   31.63366 (190) obj. 26739.4 iterations 38
Cbc0038I Pass   7: suminf.   22.73043 (155) obj. 37363.3 iterations 82
Cbc0038I Pass 

# Problem 5

In [4]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[1+abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )
    @variable(myModel, z[1:h] >= 0 )

    @objective(myModel, Min, sum(z[j] for j=1:h) )

    #introduce z for the absolute value of R-H-CHD
    @constraint(myModel, [j=1:h],z[j]>=R[j]-H[j]-10 )
    @constraint(myModel, [j=1:h],z[j]>=-R[j]+H[j]+10 )
    
    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    # constraints for x
    @constraint(myModel, x[1] + x[2] <= 1)
    @constraint(myModel, [j=2:h-1], x[j-1] + x[j] <= 1)
    @constraint(myModel, x[h-1] + x[h] <= 1)
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -seconds 20 -solve -quit (default strategy 1)
seconds was changed from 1e+100 to 20
Continuous objective value is 23.8466 - 0.01 seconds
Cgl0003I 0 fixed, 331 tightened bounds, 0 strengthened rows, 773 substitutions
Cgl0003I 0 fixed, 68 tightened bounds, 0 strengthened rows, 3 substitutions
Cgl0003I 0 fixed, 12 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 787 rows, 788 columns (234 integer (234 of which binary)) and 2504 elements
Cbc0038I Initial state - 227 integers unsatisfied sum - 42.7643
Cbc0038I Pass   1: suminf.   42.59245 (226) obj. 10872.2 iterations 30
Cbc0038I Pass   2: suminf.   37.51830 (207) obj. 12143.3 iterations 41
Cbc0038I Pass   3: suminf.   37.51830 (207) obj. 12143.3 iterations 7
Cbc0038I Pass   4: suminf.   27.53663 (168) obj. 15455.4 iterations 88
Cbc0038I Pass   5: suminf.   27.53663 (168) obj. 15455.4 iterations 5
Cbc0

# Problem 6

In [19]:
using GLPK, JuMP, SparseArrays

K_settings = [
    300 140 40;
    500 230 60;
    1000 400 70
]

function constructA_variable(H, K_settings, x_settings)
    h = length(H)
    k = size(K_settings, 2)
    I = Int[]
    J = Int[]
    V = JuMP.GenericAffExpr{Float64, JuMP.VariableRef}[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                for setting = 1:size(K_settings, 1)
                    if setting == 1
                        push!(I, i)
                        push!(J, j)
                        push!(V, K_settings[setting, 1+abs(diff)] * x_settings[j, setting])
                    else
                        push!(I,i)
                        push!(J,j)
                        push!(V, K_settings[setting, 1+abs(diff)] * x_settings[j, setting])
                    end
                end
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP_dial_yield(H, K_settings)
    h = length(H)
    num_settings = size(K_settings, 1)
    myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "seconds", 20)

    @variable(myModel, x_settings[1:h, 1:num_settings], Bin)
    @variable(myModel, R[1:h] >= 0)
    @variable(myModel, z[1:h] >= 0)

    @objective(myModel, Min, sum(z[j] for j=1:h))

    @constraint(myModel, [j=1:h], z[j] >= R[j] - H[j] - 10)
    @constraint(myModel, [j=1:h], z[j] >= -R[j] + H[j] + 10)
    @constraint(myModel, [j=1:h], R[j] >= H[j] + 10)

    # Each location can only have one setting
    @constraint(myModel, [j=1:h], sum(x_settings[j, :]) == 1)

    # No adjacent bombs
    @constraint(myModel, x_settings[1, :] + x_settings[2, :] .<= 1)
    @constraint(myModel, [j=2:h-1], x_settings[j-1, :] + x_settings[j, :] .<= 1)
    @constraint(myModel, x_settings[h-1, :] + x_settings[h, :] .<= 1)

    # R[i] = Sum of the effects of all the bombs
    @constraint(myModel, [i=1:h], R[i] == sum(constructA_variable(H, K_settings, x_settings)[i, :]))

    optimize!(myModel)
    println("hello")
    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x_settings = ", JuMP.value.(x_settings))
        println("sum of x_settings = ", sum(JuMP.value.(x_settings), dims=1))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("sum of x_settings = ", sum(JuMP.value.(x_settings), dims=1))
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x_settings = ", JuMP.value.(x_settings))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
        println("Bounds: ", JuMP.objective_bound(myModel))
    end
end

solveIP_dial_yield(H, K_settings)

hello
Optimal solution found
sum of x_settings = [159.0 158.0 0.0]
Objective value: 304546.0
x_settings = [1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0